In [ ]:
# --- useful when the package is not yet installed
import sys
sys.path.insert(0, "../")
# ---

# imports
from reform import simu_utils
from simtk.openmm import app
from simtk import unit
import numpy as np

In [ ]:
# simulation setup
pdb_path = "../reform/tests/spep_0000.pdb"

TIME_STEP = 2   # in fs
SIMU_TIME = 50  # in ps
RECORDING_INTERVAL = 1   # in ps
EXCHANGE_INTERVAL = 0.2  # in ps; or 0. when you don't need the exchanges

temps_intended = [300., 350.] # in K

OUTPUT_PATH = "/tmp/00100.npy"

In [ ]:
# prepare the MultiTSimulation
n_replicas = len(temps_intended)

# --- routinary OpenMM system setup, here it's for a vacuum system of ala2
pdb = app.PDBFile(pdb_path)
ff = app.ForceField("amber99sbildn.xml")
system = ff.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds,
                         hydrogenMass=4 * unit.amu)
integrator_params = {"integrator": "Langevin", "friction_in_inv_ps": 1.0, "time_step_in_fs": 2.0}
# ---

simu = simu_utils.MultiTSimulation(system, temps_intended, interface="single_threaded",
                                   integrator_params=integrator_params, verbose=False)
simu.set_positions([pdb.getPositions()] * n_replicas)
simu.minimize_energy()
simu.set_velocities_to_temp()
simu.run(2000)  # pre-equilibration
_ = simu.reset_step_counter()

In [ ]:
# install the simulation hooks for recording and exchange
simu_steps = simu_utils.recording_hook_setup(simu=simu, simu_time=SIMU_TIME,
                                             recording_interval=RECORDING_INTERVAL,
                                             output_path=OUTPUT_PATH,
                                             exchange_interval=EXCHANGE_INTERVAL)

In [ ]:
# a timed run
from timeit import default_timer as timer
start = timer()
simu.run(simu_steps)
stop = timer()
print("Production run of 50 ps takes {:.2f} seconds.".format(stop - start))

In [ ]:
# check the output npy file
# dimensions [N_replicas, N_frames, N_atoms, N_dims=3]
np.load(OUTPUT_PATH).shape